配分シミュレーション時に目的地分布を与えるためのコード

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# 最終形コード

生成するagentの数を与えたらagentごとの目的地を与える関数
降車位置は配分コードのところでもっと適当に与える

In [38]:
def generate_destination(num):
    #目的地の選択確率を表す辞書
    d_dic = {7.0: 0.3642213642213642, #その他
    1.0: 0.2985842985842986, #スクランブルスクエア
    2.0: 0.13256113256113256, #ヒカリエ
    5.0: 0.1093951093951094, #マークシティ
    6.0: 0.043758043758043756, #渋谷ストリーム
    4.0: 0.036036036036036036, #渋谷フクラス
    3.0: 0.015444015444015444 #渋谷109
    }
    #選択肢が7の時に適当に選択する
    #選択確率の辞書
    place_dic = {
    533935854: 0.2,
    533935863: 0.2,
    533935952: 0.4,
    533935961: 0.3
    }
    #長方形の頂点
    latlon_dic = {
    533935854: [[35.65417, 35.65833], [139.69375, 139.70000]],
    533935863: [[35.65417, 35.65833], [139.70000, 139.70625]],
    533935952: [[35.65833, 35.66250], [139.69375, 139.70000]],
    533935961: [[35.65833, 35.66250], [139.70000, 139.70625]],
    }
    choices = list(d_dic.keys())
    probabilities = list(d_dic.values())
    selections = random.choices(choices, probabilities, k=num)
    lis = []
    for item in selections:
        if item != 7:
            lat = 0
            lon = 0
        else:
            place = random.choices(list(place_dic.keys()), list(place_dic.values()), k=1)[0]
            lat = random.uniform(latlon_dic[place][0][0], latlon_dic[place][0][1])
            lon = random.uniform(latlon_dic[place][1][0], latlon_dic[place][1][1])
        lis.append([item, lat, lon])
    df = pd.DataFrame(lis, columns = ['d', 'lat', 'lon'])
    return df

In [39]:
import os 

base_path = '/Users/takahiromatsunaga/res2023'
point_sign_rate = 0
target_time = 10 # 固定
split_time = 10 # 可視化のとき何分間を可視化させるか

df_macrores = pd.read_csv(os.path.join(base_path, f'results/macro_result/macro_result0606/point_time_{point_sign_rate}.csv'))
output_path = os.path.join(base_path, 'PPcameraTG/PP/PP_webq/destination_assignment/dest_assign_res')
# /Users/takahiromatsunaga/res2023/results/macro_result/macro_result0606/point_time_40.csv

# df_macroresから渋谷駅到着人数を取得
arrival = df_macrores.loc[target_time, 'shibuya_arrive'] # 0時から始まってるのでそのままindexになる
print(int(arrival*(split_time/60)))
df_destassign_res = generate_destination(int(arrival*(split_time/60)))

df_destassign_res.to_csv(os.path.join(output_path, f'destassign_{target_time}_{point_sign_rate}.csv'))

3545


適当に降車位置を割り付ける

In [40]:
car_score_dict = {
    1: 0.11888111888111888, 
    2: 0.027972027972027972,
    3: 0.03636363636363636, 
    4: 0.04755244755244755, 
    5: 0.026573426573426574, 
    6: 0.005594405594405594,
    7: 0.00,
    8: 0.0979020979020979, 
    9: 0.16223776223776223, 
    10: 0.05874125874125874, 
    11: 0.07972027972027972, 
    12: 0.04755244755244755, 
    13: 0.057342657342657345,
    14: 0.11888111888111888, 
    15: 0.11468531468531469
    }

P_accum = np.zeros(len(car_score_dict)) # 累積確率
car_score_value = list(car_score_dict.values())
for i in range(len(car_score_dict)):
    if i == 0:
        P_accum[i] = car_score_value[i]
    else:
        P_accum[i] = P_accum[i-1] + car_score_value[i]
print(P_accum)

[0.11888112 0.14685315 0.18321678 0.23076923 0.25734266 0.26293706
 0.26293706 0.36083916 0.52307692 0.58181818 0.66153846 0.70909091
 0.76643357 0.88531469 1.        ]


In [41]:
point_sign_rate = 0
target_time = 10 # これは基本固定
df_destination_res = pd.read_csv(os.path.join(base_path, f'PPcameraTG/PP/PP_webq/destination_assignment/dest_assign_res/destassign_{target_time}_{point_sign_rate}.csv'))
df_destination_car_res = df_destination_res.copy()
df_destination_car_res['car'] = 0
for i in range(len(df_destination_car_res)):
    rand = np.random.rand()
    if rand <= P_accum[0]:
        df_destination_car_res.loc[i, 'car'] = 1
    else:
        for j in range(1, len(P_accum)):
            if P_accum[j-1] <= rand <= P_accum[j]:
                df_destination_car_res.loc[i, 'car'] = j+1
            else:
                continue
print(df_destination_car_res.head(15))

df_destination_car_res.to_csv(os.path.join(base_path, f'PPcameraTG/PP/PP_webq/destination_assignment/dest_assign_res/destcar_assign_{target_time}_{point_sign_rate}.csv'))


    Unnamed: 0    d        lat         lon  car
0            0  2.0   0.000000    0.000000   11
1            1  4.0   0.000000    0.000000   14
2            2  2.0   0.000000    0.000000    8
3            3  7.0  35.659211  139.703830    9
4            4  7.0  35.656415  139.695672   10
5            5  1.0   0.000000    0.000000   15
6            6  7.0  35.656369  139.693889    8
7            7  7.0  35.661140  139.705588   11
8            8  7.0  35.657298  139.698832   11
9            9  4.0   0.000000    0.000000   13
10          10  1.0   0.000000    0.000000   10
11          11  1.0   0.000000    0.000000    1
12          12  7.0  35.657942  139.705972   11
13          13  7.0  35.657441  139.693912    1
14          14  7.0  35.661619  139.705611    4


In [ ]:
df_destination_car_res.to_csv(os.path.join())

In [ ]:
car_score_dict = {
    1: 0.11888111888111888, 
    2: 0.027972027972027972,
    3: 0.03636363636363636, 
    4: 0.04755244755244755, 
    5: 0.026573426573426574, 
    6: 0.005594405594405594,
    7: 0.00,
    8: 0.0979020979020979, 
    9: 0.16223776223776223, 
    10: 0.05874125874125874, 
    11: 0.07972027972027972, 
    12: 0.04755244755244755, 
    13: 0.057342657342657345,
    14: 0.11888111888111888, 
    15: 0.11468531468531469
    }




# 作業

In [7]:
#オリジナルデータは変えない
original_data = pd.read_csv('diary_cleaned.csv', index_col = 0)

In [5]:
original_data 

,id,day,time,purpose,d,o,d_line,d_car,staire_to_gate,d_gate,o_line,o_gate,staire_to_home,o_car
0,26,12/19,10:00-11:00,2.0,7.0,5.0,1.0,4.0,12,4.0,5.0,NaN,NaN,NaN
1,20,12/21,13:00-14:00,11.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,12/20,14:00-15:00,NaN,7.0,7.0,1.0,3.0,12,4.0,2.0,4.0,5,2.0
3,244,12/21,18:00-19:00,4.0,NaN,NaN,1.0,4.0,12,5.0,9.0,NaN,NaN,NaN
4,120,12/19,6:00-7:00,2.0,NaN,NaN,1.0,1.0,12,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,78,01/28,21:00-22:00,4.0,1.0,1.0,1.0,8.0,6,1.0,8.0,NaN,NaN,NaN
1575,179,01/27,17:00-18:00,11.0,NaN,NaN,1.0,8.0,7,2.0,11.0,NaN,NaN,NaN
1576,105,01/30,8:00-9:00,11.0,NaN,NaN,9.0,NaN,NaN,NaN,2.0,5.0,NaN,1.0
1577,73,01/27,18:00-19:00,4.0,1.0,1.0,4.0,3.0,15,1.0,8.0,NaN,NaN,NaN


In [8]:
data =  original_data.dropna(subset = ['d'])

In [11]:
#目的地の割合を取得
d_dic = data['d'].value_counts(normalize=True).to_dict()

{7.0: 0.3642213642213642,
 1.0: 0.2985842985842986,
 2.0: 0.13256113256113256,
 5.0: 0.1093951093951094,
 6.0: 0.043758043758043756,
 4.0: 0.036036036036036036,
 3.0: 0.015444015444015444}

In [24]:
vertax_df = pd.read_csv('vertax.csv', index_col = 0)
#絞る
vertax_df = vertax_df[vertax_df.index.isin([533935952, 533935961, 533935854, 533935863])]
vertax_df_cleaned = vertax_df[['latitude', 'longitude']]

In [26]:
vertax_df_cleaned

,latitude,longitude
MESH_ID,,
533935854,35.65417,139.69375
533935854,35.65833,139.69375
533935854,35.65833,139.70000
533935854,35.65417,139.70000
533935854,35.65417,139.69375
533935863,35.65417,139.70000
533935863,35.65833,139.70000
533935863,35.65833,139.70625
533935863,35.65417,139.70625


In [ ]:
#選択肢が7の時に適当に選択する
#選択確率の辞書
place_dic = {
533935854: 0.2,
533935863: 0.2,
533935952: 0.4,
533935961: 0.3
}
#長方形の頂点
latlon_dic = {
533935854: [[35.65417, 35.65833], [139.69375, 139.70000]],
533935863: [[35.65417, 35.65833], [139.70000, 139.70625]],
533935952: [[35.65833, 35.66250], [139.69375, 139.70000]],
533935961: [[35.65833, 35.66250], [139.70000, 139.70625]],
}





In [29]:
def generate_destination(num):
    #目的地の選択確率を表す辞書
    d_dic = {7.0: 0.3642213642213642, #その他
    1.0: 0.2985842985842986, #スクランブルスクエア
    2.0: 0.13256113256113256, #ヒカリエ
    5.0: 0.1093951093951094, #マークシティ
    6.0: 0.043758043758043756, #渋谷ストリーム
    4.0: 0.036036036036036036, #渋谷フクラス
    3.0: 0.015444015444015444 #渋谷109
    }
    #選択肢が7の時に適当に選択する
    #選択確率の辞書
    place_dic = {
    533935854: 0.2,
    533935863: 0.2,
    533935952: 0.4,
    533935961: 0.3
    }
    #長方形の頂点
    latlon_dic = {
    533935854: [[35.65417, 35.65833], [139.69375, 139.70000]],
    533935863: [[35.65417, 35.65833], [139.70000, 139.70625]],
    533935952: [[35.65833, 35.66250], [139.69375, 139.70000]],
    533935961: [[35.65833, 35.66250], [139.70000, 139.70625]],
    }
    choices = list(d_dic.keys())
    probabilities = list(d_dic.values())
    selections = random.choices(choices, probabilities, k=num)
    lis = []
    for item in selections:
        if item != 7:
            lat = 0
            lon = 0
        else:
            place = random.choices(list(place_dic.keys()), list(place_dic.values()), k=1)[0]
            lat = random.uniform(latlon_dic[place][0][0], latlon_dic[place][0][1])
            lon = random.uniform(latlon_dic[place][1][0], latlon_dic[place][1][1])
        lis.append([item, lat, lon])
    df = pd.DataFrame(lis, columns = ['d', 'latitude', 'longitude'])
    return df

In [32]:
generate_destination(100)

,d,latitude,longitude
0,3.0,0.000000,0.000000
1,5.0,0.000000,0.000000
2,6.0,0.000000,0.000000
3,6.0,0.000000,0.000000
4,7.0,35.658402,139.698647
...,...,...,...
95,7.0,35.659370,139.696028
96,7.0,35.661195,139.694720
97,1.0,0.000000,0.000000
98,1.0,0.000000,0.000000


diary dataのdcar_nodeから割合を見る

In [17]:
df_diary = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary_cleaned_dcar_latlon_point6.csv')
# 降車位置の割合を取得
d_dic2 = df_diary['dcar_node'].value_counts(normalize=True).to_dict()
print(d_dic2)

{0: 0.3299556681443952, 9: 0.08929702343255225, 15: 0.07473084230525649, 1: 0.06649778340721976, 14: 0.06586447118429385, 8: 0.06016466117796074, 11: 0.0506649778340722, 13: 0.03926535782140595, 10: 0.037365421152628246, 12: 0.028499050031665613, 4: 0.024699176694110196, 3: 0.02089930335655478, 5: 0.018999366687777075, 2: 0.017099430018999367, 22: 0.013299556681443952, 21: 0.011399620012666244, 18: 0.011399620012666244, 20: 0.010766307789740342, 19: 0.008866371120962635, 17: 0.006333122229259025, 16: 0.005699810006333122, 23: 0.00506649778340722, 6: 0.00253324889170361, 7: 0.0006333122229259025}


1月以降のデータ

In [19]:
from datetime import datetime as dt 
df_diary = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary_cleaned_dcar_latlon_point6.csv')
df_diary2 = df_diary.copy()
df_diary2['day'] = pd.to_datetime(df_diary2['day'], format='%m/%d')

# 1月の日付のみを抽出
df_january = df_diary2[df_diary2['day'].dt.month == 1]
df_january_nonzero_dcar = df_january[df_january['dcar_node'] != 0]

# 降車位置の割合を取得
d_dic2 = df_january_nonzero_dcar['dcar_node'].value_counts(normalize=True).to_dict()
print(d_dic2)

{9: 0.16223776223776223, 1: 0.11888111888111888, 14: 0.11888111888111888, 15: 0.11468531468531469, 8: 0.0979020979020979, 11: 0.07972027972027972, 10: 0.05874125874125874, 13: 0.057342657342657345, 4: 0.04755244755244755, 12: 0.04755244755244755, 3: 0.03636363636363636, 2: 0.027972027972027972, 5: 0.026573426573426574, 6: 0.005594405594405594}
